In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from PIL import Image
from IPython.core.display import HTML
from functools import partial
import cv2

from visprog.engine.utils import ProgramGenerator, ProgramInterpreter
from visprog.prompts.clevrer import create_prompt
import base64
import numpy as np
from tqdm import tqdm

[2024-01-22 11:30:03,960] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
module_list = ["LOC", "TRACK", "GRAY", "RED", "BLUE", "GREEN", "CUBE", "SPHERE", "RUBBER", "LEFT", "TOP", "LEFTOF", "FRONTOF", "EVENT", "BEFORE", "EVAL", "RESULT"]
use_precomputed = True
interpreter = ProgramInterpreter(dataset='clevrer', use_precomputed=use_precomputed, module_list=module_list)


Registering LOC step
Registering TRACK step
Registering LEFT step
Registering RIGHT step
Registering TOP step
Registering BOTTOM step
Registering GRAY step
Registering RED step
Registering BLUE step
Registering GREEN step
Registering BROWN step
Registering PURPLE step
Registering CYAN step
Registering YELLOW step
Registering CUBE step
Registering SPHERE step
Registering CYLINDER step
Registering RUBBER step
Registering METAL step
Registering LEFTOF step
Registering RIGHTOF step
Registering FRONTOF step
Registering BEHIND step
Registering EVENT step
Registering BEFORE step
Registering EVAL step
Registering RESULT step


In [4]:
prog = """OBJ0=LOC(video=VIDEO,object='object')
OBJT=TRACK(object=OBJ0)
PRED0=RED(object=OBJT,var='o0')
PRED1=RUBBER(object=OBJT,var='o0')
PRED2=TOP(object=OBJT,var='o1')
PRED3=FRONTOF(object1=OBJT,var1='o0',object2=OBJT,var2='o1')
EVENT0=EVENT(predicates=[PRED0,PRED1,PRED2,PRED3],min_duration=1)
PRED4=FRONTOF(object1=OBJT,var1='o1',object2=OBJT,var2='o0')
EVENT1=EVENT(predicates=[PRED4],min_duration=1)
EVENT2=BEFORE(event1=EVENT0, event2=EVENT1)
PRED5=TOP(object=OBJT,var='o2')
PRED6=LEFT(object=OBJT,var='o2')
EVENT3=EVENT(predicates=[PRED5,PRED6],min_duration=25)
EVENT4=BEFORE(event1=EVENT2, event2=EVENT3)
ANSWER0=EVAL(expr="'yes' if len({EVENT4}) else 'no'")
FINAL_RESULT=RESULT(var=ANSWER0)"""

In [6]:
output = []
for vid in tqdm(range(10000)):
    video = []
    if not use_precomputed:
        cap = cv2.VideoCapture(
            os.path.join(
                '/gscratch/balazinska/enhaoz/VOCAL-UDF/data',
                'clevrer',
                f'video_{str(vid//1000*1000).zfill(5)}-{str((vid//1000+1)*1000).zfill(5)}',
                f"video_{str(vid).zfill(5)}.mp4"
            )
        )
        fid = 0
        while True:
            ret, frame = cap.read()  # Read the next frame from the video
            if not ret:
                break  # Break the loop if there are no more frames

            # Convert the frame from BGR (OpenCV format) to RGB (PIL format)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Convert the frame to a PIL Image and append to the list
            image = Image.fromarray(frame)
            image.thumbnail((640,640),Image.Resampling.LANCZOS)
            video.append(image)
            fid += 1
    init_state = dict(
        VIDEO=video,
        vid=vid
    )
    result, prog_state = interpreter.execute(prog,init_state,inspect=False)
    if result == 'yes':
        output.append(vid)
print(output)


  0%|                                                  | 0/10000 [00:00<?, ?it/s]

LOC
TRACK
RED
RUBBER
TOP
FRONTOF
EVENT
FRONTOF
EVENT



  0%|                                        | 1/10000 [00:01<2:52:44,  1.04s/it]

BEFORE
TOP
LEFT
EVENT
BEFORE
EVAL
RESULT
LOC



  0%|                                        | 2/10000 [00:01<1:30:12,  1.85it/s]

...

BEFORE
EVAL
RESULT
LOC
TRACK
RED
RUBBER
TOP
FRONTOF
EVENT
FRONTOF
EVENT
BEFORE
TOP
LEFT
EVENT
BEFORE
EVAL
RESULT
LOC
TRACK
RED
RUBBER
TOP
FRONTOF
EVENT
FRONTOF
EVENT
BEFORE
TOP
LEFT
EVENT



100%|██████████████████████████████████████▉| 9998/10000 [13:39<00:00,  9.81it/s]

BEFORE
EVAL
RESULT
LOC
TRACK
RED
RUBBER
TOP
FRONTOF
EVENT
FRONTOF
EVENT
BEFORE
TOP
LEFT
EVENT
BEFORE
EVAL
RESULT
LOC
TRACK
RED
RUBBER
TOP
FRONTOF
EVENT
FRONTOF
EVENT


100%|██████████████████████████████████████| 10000/10000 [13:39<00:00, 12.20it/s]

BEFORE
TOP
LEFT
EVENT
BEFORE
EVAL
RESULT
[2, 6, 41, 83, 95, 109, 111, 114, 126, 151, 154, 163, 165, 207, 214, 223, 248, 257, 315, 317, 346, 351, 369, 406, 424, 431, 493, 501, 510, 530, 546, 555, 591, 609, 619, 632, 650, 673, 688, 697, 698, 711, 730, 734, 773, 777, 805, 808, 818, 837, 843, 868, 879, 883, 900, 906, 923, 935, 947, 948, 954, 958, 971, 987, 999, 1015, 1016, 1031, 1038, 1060, 1073, 1076, 1086, 1091, 1111, 1117, 1145, 1147, 1173, 1182, 1191, 1200, 1201, 1213, 1219, 1226, 1227, 1238, 1267, 1272, 1279, 1287, 1289, 1290, 1338, 1354, 1361, 1363, 1384, 1391, 1434, 1439, 1468, 1479, 1481, 1487, 1489, 1501, 1540, 1542, 1543, 1553, 1565, 1603, 1612, 1615, 1640, 1709, 1711, 1712, 1757, 1763, 1776, 1777, 1779, 1799, 1814, 1816, 1858, 1860, 1880, 1896, 1965, 1998, 2003, 2006, 2034, 2045, 2061, 2062, 2070, 2075, 2097, 2118, 2140, 2149, 2222, 2223, 2252, 2258, 2261, 2266, 2270, 2274, 2282, 2304, 2316, 2333, 2347, 2370, 2371, 2394, 2424, 2433, 2470, 2488, 2492, 2495, 2502, 2514, 2515, 2557